In [613]:
import re

import yaml
import json
import pandas as pd

from langchain_community.document_loaders import UnstructuredPDFLoader, UnstructuredWordDocumentLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

from atomic_sovus.preprocess import filter_text_documents

In [2]:
import glob

In [611]:
titles_all_json = {
    "Инструкция Интеграция с ЕОСДО v5.pdf": {
        "name": "Инструкция пользователей «Интеграция 1С ERP 2.0: Цифровой Росатом с ЕОСДО»",
        "titles": {
            "СОДЕРЖАНИЕ ПЕРЕЧЕНЬ СОКРАЩЕНИЙ, ТЕРМИНОВ И ОПРЕДЕЛЕНИЙ": 4,
            "ЦЕЛИ И ЗАДАЧИ НАСТОЯЩЕЙ ИНСТУКЦИИ": 5,
            "Описание интеграционных потоков и их с взаимосвязи с функциональными бизнес- процессами.": 6,
            "1.1 Интеграционный сценарий «Система электронного архива (СЭА)» (10-й сценарий ЕОСДО)": 8,
            "1.2 Интеграционный сценарий «Массовая выгрузка документов из ЕОСДО»": 10,
            "1.3 Передача платежных поручений (мемориальных ордеров) в ЕОСДО": 10,
            "Описание этапов интеграции": 12,
            "2.1 Интеграционный сценарий «Система электронного архива (СЭА)» (10-й сценарий ЕОСДО)": 12,
            "Предприятие: регистрация первичного документа и проведение в управленческом учете": 12,
            "Предприятие: Отправка пакета первичной документации посредством Captiva": 18,
            "Предприятие / Бухгалтер: Проверка сделки в ЕОСДО": 23,
            "Предприятие / Бухгалтер: Сверка первичных документов из 1С ERP 2.0: Цифровой Росатом": 24,
            "2.2 Интеграционный сценарий «Массовая выгрузка документов из ЕОСДО»": 26,
            "Бухгалтер по учету налоговых обязательств и формированию отчетности (НП) МФ ОЦО / Главный бухгалтер: Формирование реестра документов, которые являются основанием для налоговой декларации по налогу на прибыль": 27,
            "Бухгалтер по учету НДС / Главный бухгалтер: Формирование реестра документов, которые являются основанием для налоговой декларации по НДС": 29,
            "Бухгалтер по учету ТМЦ / Бухгалтер по учету НДС / Бухгалтер по учету операций с ВНА / Главный бухгалтер: Формирование реестра документов, полученных от контрагентов, регистрация которых в регламентированном учете происходит по 41-му счету.": 30,
            "Бухгалтер по учету банковских операций / Главный бухгалтер: Формирование реестра документов платежных документов": 31,
            "Бухгалтер по учету средств целевого финансирования / Главный бухгалтер: Формирование реестра документов по использованию средств целевого финансирования": 33,
            "2.3 Передача платежных поручений (мемориальных ордеров) в ЕОСДО": 34,
            "ПРЕДУПРЕЖДЕНИЯ, СООБЩЕНИЯ ОБ ОШИБКАХ": 35
        }
    },
    "Инструкция D-1C1-1.10.39 Сверка взаиморасчетов с контрагентами.pdf": {
        "name": "Сверка взаиморасчетов с контрагентами",
        "titles": {
            "Глоссарий": 7,
            "1. Цели и задачи настоящей инструкции": 8,
            "2. Описание этапов бизнес-процесса": 9,
            "2.1. Последовательность этапов бизнес-процесса": 9,
            "2.2. Описание этапов бизнес-процесса": 9,
            "2.2.1. Экономист: Предварительные настройки системы": 9,
            "2.2.2. Бухгалтер по сверке взаимных расчетов: Выбор документа или отчета в ПП 1С для оформления результатов": 12,
            "2.2.3. Бухгалтер по сверке взаимных расчетов: Формирование документа «Сверка взаиморасчета»": 18,
            "2.2.4. Особенности отражения сверки взаиморасчетов в рамках электронного документа": 27,
            "2.2.4.1. Работа с документом «Сверка взаиморасчетов» в рамках ЮЗ ЭДО.": 27,
            "2.2.4.2. Подписание ЭД «Акт сверки взаиморасчетов» и «Протокол расхождений» (Подписант ЮЗ ЭДО)": 30,
            "2.2.4.3. Получение документа и сверка взаиморасчетов на стороне контрагента (Бухгалтер по сверке взаимных расчетов, Менеджер ЮЗ ЭДО)": 34,
            "2.2.4.4. Настройка оповещений о получении ЭД и об изменении его статусов (Бухгалтер по сверке взаимных расчетов)": 35,
            "2.2.5. Бухгалтер по сверке взаимных расчетов: Отражение в учете расхождений во взаиморасчетах с контрагентами": 36,
            "2.2.5.1 Взаимозачет задолженности": 36,
            "2.2.5.2. Корректировка данных с помощью документа «Корректировка Задолженности»": 36,
            "2.2.6. Бухгалтер по сверке взаимных расчетов: Формирование отчета по дебиторской и кредиторской задолженности": 51
        }
    },
    "Инструкция D-1C1-1.10.25 Учет расчетов по заработной плате3 (НФ).pdf": {
        "name": "Учет расчетов по заработной плате",
        "titles": {
            "1. Цели и задачи настоящей инструкции": 5,
            "2. Описание этапов бизнес-процесса": 6,
            "2.1 Последовательность этапов бизнес-процесса": 6,
            "2.2 Описание этапов бизнес-процесса": 6,
            "2.2.1 Загрузка данных из системы расчета заработной платы": 6,
            "2.2.2 Сверка загруженных данных в 1С с данными ИАСУП": 22,
            "2.2.3 Передача резервов между филиалами": 24
        }
    },
    "Инструкция пользователей «D-1C1-1.22.01 Формирование графиков платежей, поступления ДС, поставок, отгрузок».pdf": {
        "name": "Формирование графиков платежей, поступления ДС, поставок, отгрузок",
        "titles": {
            "ПЕРЕЧЕНЬ СОКРАЩЕНИЙ, ТЕРМИНОВ И ОПРЕДЕЛЕНИЙ": 4,
            "ЦЕЛИ И ЗАДАЧИ НАСТОЯЩЕЙ ИНСТРУКЦИИ": 5,
            "1 ОПИСАНИЕ ЭТАПОВ БИЗНЕС-ПРОЦЕССА": 6,
            "1.1 Последовательность этапов бизнес-процесса": 6,
            "1.2 Описание этапов бизнес-процесса": 6,
            "1.2.1 Заполнение графика платежей, Менеджер по продажам, Менеджер по закупкам": 6,
            "Заполнение графика платежей в Заказе клиента и Заказе поставщику": 7,
            "Заполнение графика платежей в документах «Реализация товаров и услуг» и «Приобретение товаров и услуг»": 15,
            "Заполнение графика платежей в документах «Реализация услуг и прочих активов», «Приобретение услуг и прочих активов», первичных документах": 19,
            "Заполнение дат планового погашения авансов в документах «Списание безналичных ДС», «Поступление безналичных ДС»": 22,
            "1.2.2 Корректировка графика и дат планового погашения, Менеджер по продажам, Менеджер по закупкам": 29,
            "1.2.3 Формирование отчетов для анализа дат планового погашения": 31,
            "1.2.4 Привязка фактических данных оплаты к графикам, Менеджер по закупкам, Менеджер по продажам": 33
        }
    },
    "Инструкция администратора Настройка ограничения прав доступа пользователей по ЦФО и статьям бюджетов для АО Гринатом.pdf": {
        "name": "Настройка ограничения прав доступа пользователей по ЦФО и статьям бюджетов для АО «Гринатом»",
        "titles": {
            "Перечень сокращений, терминов и определений": 6,
            "1. Цели и задачи настоящей инструкции": 7,
            "2. Последовательность и описание этапов бизнес-процесса": 7,
            "2.1. Последовательность выполнения бизнес-процессов": 7,
            "2.2. Описание этапов бизнес-процесса": 7,
            "Администратор: включение механизма разграничения доступа на уровне записей по ЦФО и статьям бюджетов": 7,
            "Администратор: настройка групп доступа пользователей по ЦФО и статьям бюджетов": 8
        }
    },
    "Интеграция РЦК_v2 (НФ).pdf": {
        "name": "РЦК",
        "titles": {
            "СОДЕРЖАНИЕ ПЕРЕЧЕНЬ СОКРАЩЕНИЙ, ТЕРМИНОВ И ОПРЕДЕЛЕНИЙ": 4,
            "ЦЕЛИ И ЗАДАЧИ НАСТОЯЩЕЙ ИНСТУКЦИИ": 5,
            "1 ОПИСАНИЕ ЭТАПОВ БИЗНЕС ПРОЦЕССОВ": 5,
            "Последовательность этапов бизнес-процесса": 5,
            "Профили групп доступа": 5,
            "Описание этапов бизнес-процесса": 6,
            "Казначей: Формирование списка документов для выгрузки": 6,
            "Отправка подтверждающих документов для валютных сделок": 10,
            "Отправка документов в РЦК для расчета прогноза ликвидности": 11,
            "2 ПРЕДУПРЕЖДЕНИЯ, СООБЩЕНИЯ ОБ ОШИБКАХ": 12
        }
    },
    "Инструкция D-1C1-1.01.01 Универсальное согласование документов _v6.pdf": {
        "name": "Универсальное согласование (согласование документов)",
        "titles": {
            "ПЕРЕЧЕНЬ СОКРАЩЕНИЙ, ТЕРМИНОВ И ОПРЕДЕЛЕНИЙ": 6,
            "ЦЕЛИ И ЗАДАЧИ НАСТОЯЩЕЙ ИНСТРУКЦИИ": 7,
            "1 ОПИСАНИЕ ЭТАПОВ БИЗНЕС-ПРОЦЕССА": 8,
            "1.1 Последовательность этапов бизнес-процесса": 8,
            "1.2 Роли и полномочия пользователей для согласования объектов": 8,
            "1.3 Этапы бизнес-процесса": 9,
            "1.3.1 Процесс согласования объектов системы": 9,
            "1.3.2 Обработка «Мои задачи и оповещения»": 14,
            "1.3.3 Назначение заместителей и дополнительных согласующих": 20,
            "1.3.4 Настройка оповещений на электронную почту": 23,
            "1.4 Корректировка объектов согласования": 25,
            "1.4.1. Объект согласования в статусе «Черновик», «Отклонен» или «Отправлен на уточнение»": 25,
            "1.4.2. Объект согласования в статусе «На утверждении»": 25,
            "1.4.3. Объект согласования в статусе «Утвержден»": 27,
            "1.4.4. Документ проверен бухгалтером ОЦО": 29,
            "2 ПРЕДУПРЕЖДЕНИЯ, СООБЩЕНИЯ ОБ ОШИБКАХ": 30,
            "Приложение 1. Настройка доступности реквизитов в утвержденных документах": 35
        }
    },
    "Инструкция D-1C1-1.03.04 Выпуск продукции услуг работ собственным производством (НФ) (5) (2).pdf": {
        "name": "Выпуск продукции, услуг, работ собственным производством",
        "titles": {
            "ПЕРЕЧЕНЬ СОКРАЩЕНИЙ, ТЕРМИНОВ И ОПРЕДЕЛЕНИЙ": 4,
            "ЦЕЛИ И ЗАДАЧИ НАСТОЯЩЕЙ ИНСТУКЦИИ": 5,
            "1. ОПИСАНИЕ ЭТАПОВ БИЗНЕС-ПРОЦЕССА": 6,
            "1.1 Последовательность этапов бизнес-процесса": 6,
            "1.2 Профили групп доступа": 6,
            "1.3 Описание бизнес-процесса": 7,
            "1.3.1 Выпуск готовой продукции работ, услуг (Экономист производства)": 7,
            "1.3.1.1 Оформление выпуска продукции через документ «Производство без заказа» (попроцессный или попередельный вид производства)": 8,
            "1.3.1.2 Оформление выпуска продукции через документ «Заказ на производство» (позаказный вид производства)": 12,
            "1.3.2 Определение направления выпуска ГП, работ, услуг (Экономист производства)": 19,
            "1.3.3 Оформление выпуска обслуживающих производств (Экономист производства)": 21,
            "1.3.4 Проверка документов выпуска и формирование проводок (Бухгалтер по учету затрат и формированию себестоимости)": 24
        }
    },
    "Инструкция Учет внутрихозяйственных расчетов.pdf": {
        "name": "Учет внутрихозяйственных расчетов",
        "titles": {
            "ЦЕЛИ И ЗАДАЧИ НАСТОЯЩЕЙ ИНСТРУКЦИИ": 5,
            "ОПИСАНИЕ ЭТАПОВ БИЗНЕС-ПРОЦЕССА": 5,
            "2.1 Последовательность этапов бизнес-процесса": 5,
            "2.2 Описание этапов бизнес-процесса": 6,
            "Определение вида операции по внутрихозяйственным расчетам, Бухгалтер по учету внутрихозяйственных операций МФ ОЦО": 6,
            "Определение вида передачи по внутрихозяйственным расчетам, Бухгалтер по учету внутрихозяйственных операций МФ ОЦО": 6,
            "Определение вида приема по внутрихозяйственным расчетам, Бухгалтер по учету внутрихозяйственных операций МФ ОЦО": 7,
            "Работа с документом «Перемещение товаров», Кладовщик / МОЛ": 7,
            "2.2.4.1 Оформление в системе документа «Перемещение товаров»": 7,
            "2.2.4.2 Отражение приема ТМЦ по внутрихозяйственным расчетам": 12,
            "Работа с документом «Перемещение ОС», Кладовщик / МОЛ": 15,
            "2.2.5.1 Формирование документа «Перемещение ОС»": 15,
            "2.2.5.2 Отражение приема ОС по внутрихозяйственным расчетам": 20,
            "Перенос задолженности поставщику между филиалами": 23,
            "2.2.6.1 Перенос задолженности поставщику между филиалами вручную": 24,
            "2.2.6.1.2 Постоплата по документу «Поступление товаров и услуг»": 27,
            "2.2.6.2 Формирование документа «Взаимозачет задолженности», Ответственный за учет внутрихозяйственных операций": 29,
            "2.2.6.3 Формирование автоматического переноса задолженности в накладных и платежных документах": 33,
            "2.2.6.3.1 Перенос задолженности филиала «Поступление товаров и услуг» при наличии аванса на головной организации": 33,
            "2.2.6.3.2 Постоплата по документу «Поступление товаров и услуг»": 49,
            "2.2.6.3.4 Оформление корректировки приобретения на уменьшение закупки с последующим переносом расхождения с филиала на ЦА": 67,
            "Перенос задолженности клиента между филиалами": 73,
            "2.2.7.1 Зачет аванса в документе «Реализация товаров и услуг»": 74,
            "2.2.7.2 Постоплата по документу «Реализация товаров и услуг»": 77,
            "Передача прочих расчетов с контрагентами между филиалами, Ответственный за учет внутрихозяйственных операций": 78,
            "Перенос денежных документов с головной организации в филиал и наоборот": 84,
            "Перенос налогов между филиалом и головной организацией": 88,
            "2.2.10.1 Перенос задолженности по налогам с филиала на головную организацию": 88,
            "2.2.10.2 Перенос оплаты налогов с головной организации на филиал": 93,
            "Передача финансового результата филиала в головную организацию, Ответственный за учет внутрихозяйственных операций": 97,
            "Передача накопленных затрат с филиалов в головную организацию и наоборот, Ответственный за учет внутрихозяйственных операций": 100,
            "Перенос резервов по зарплате между филиалами, Бухгалтер по учету расчетов с персоналом": 110,
            "Перенос платежей между филиалами, Ответственный за учет внутрихозяйственных операций": 114,
            "Перенос остатков по 76-ому счету между филиалами, Ответственный за учет внутрихозяйственных операций": 119
        }
    },
    "Отражение операций по давальческой схеме через документ «Заказ давальца»_v2.pdf": {
        "name": "Отражение операций по давальческой схеме через документ «Заказ давальца»",
        "titles": {
            "ПЕРЕЧЕНЬ СОКРАЩЕНИЙ, ТЕРМИНОВ И ОПРЕДЕЛЕНИЙ": 4,
            "ЦЕЛИ И ЗАДАЧИ НАСТОЯЩЕЙ ИНСТРУКЦИИ": 5,
            "1. Описание этапов бизнес-процесса": 6,
            "1.1 Последовательность этапов бизнес-процесса": 6,
            "1.2 Профили групп доступа": 7,
            "1.3 Описание этапов бизнес-процесса": 8,
            "1.3.1 АРМ «Документы к оформлению»": 8,
            "1.3.2 Документ «Заказ давальца»": 9,
            "1.3.3 Документ «Поступление сырья от давальца»": 14,
            "1.3.4 Документ «Заказ материалов в производство»": 15,
            "1.3.5 Документ «Передача материалов в кладовую»": 17,
            "1.3.6 Распределение материалов на партии выпуска": 19,
            "1.3.7 Документ «Этап на производство»": 20,
            "1.3.8 Документ «Передача давальцу»": 22,
            "1.3.9 Документ «Отчет давальцу»": 23,
            "1.3.10 Мониторинг исполнения заказов давальцев": 26
        }
    },
    "Инструкция D-1C1-1.01.09 Формирование платежных поручений (НФ) (5) (1).pdf": {
        "name": "Формирование платежных поручений",
        "titles": {
            "СОДЕРЖАНИЕ ПЕРЕЧЕНЬ СОКРАЩЕНИЙ, ТЕРМИНОВ И ОПРЕДЕЛЕНИЙ": 4,
            "ЦЕЛИ И ЗАДАЧИ НАСТОЯЩЕЙ ИНСТРУКЦИИ": 5,
            "1 ОПИСАНИЕ ЭТАПОВ БИЗНЕС-ПРОЦЕССА": 6,
            "1.1 Последовательность этапов бизнес-процесса": 6,
            "1.2 Профили групп доступа": 6,
            "1.3 Описание этапов бизнес-процесса": 6
        }
    },
    "Инструкция D-1C1-1.04.12 Учет договоров длительного технологического цикла_v26.pdf": {
        "name": "Учет договоров длительного технологического цикла",
        "titles": {
            "ПЕРЕЧЕНЬ СОКРАЩЕНИЙ, ТЕРМИНОВ И ОПРЕДЕЛЕНИЙ": 6,
            "ЦЕЛИ И ЗАДАЧИ НАСТОЯЩЕЙ ИНСТРУКЦИИ": 8,
            "1 ОПИСАНИЕ ЭТАПОВ БИЗНЕС–ПРОЦЕССА": 9,
            "1.1 Последовательность этапов бизнес–процесса": 9,
            "1.2 Профили групп доступа": 10,
            "1.3 Описание этапов бизнес–процесса": 11,
            "1.3.1 Ведение нормативно–справочной информации (НСИ)": 11,
            "1.3.1.1 Учетная политика организации": 11,
            "1.3.1.2 Справочник «Направление деятельности»": 14,
            "1.3.1.3 Справочник «Договоры с контрагентами»": 17,
            "1.3.1.4 Документ «Заказ клиента»": 20,
            "1.3.1.5 Документ «План по обязанности к исполнению»": 22,
            "1.3.1.6 Корректировка Документа План по обязанности к исполнению": 25,
            "1.3.1.7 Документ Заказ на производство": 25,
            "1.3.1.8 Документ Этап производство": 25,
            "1.3.1.9 Документ Расчет коэффициента степени готовности": 28,
            "1.3.1.10 Реализация товаров и услуг: частичное признание": 36,
            "1.3.1.11 Корректировка реализации работ с типом «частичное признание» по договорам ДЦИ": 38,
            "1.3.1.12 Регламент работы в случае отнесения на заказ только затрат": 41,
            "1.3.1.13 Реализация работ с типом выручки «Акт выполненных работ» по договорам ДЦИ": 42,
            "1.3.1.14 Корректировка реализация работ с типом выручки «Акт выполненных работ» или «Закрывающий акт» по договорам ДЦИ": 45,
            "1.3.1.15 Переоценка валютных договор ДЦИ: выручка по ЧП и НДС": 46,
            "1.3.1.16 Обработка по созданию документов переоценки": 48,
            "1.3.1.17 Учет опоздавших расходов": 49,
            "1.3.1.18 Финальное актирование договоров ДЦИ в случае остатков по НУ": 50,
            "1.3.1.19 Оценочные обязательства по обременительным договорам ДЦИ": 53,
            "2 ОТЧЕТНОСТЬ ПО ДОГОВОРАМ ДЦИ": 54,
            "2.1 Отчет «Анализ расчет КСГ»": 54,
            "2.2 Отчет «Остатки по ДЦИ»": 54,
            "Отчет «Расчет себестоимости актов ДЦИ»": 63,
            "3 АЛГОРИТМЫ РАСЧЕТОВ": 65,
            "3.1 Алгоритмы расчета КСГ": 65,
            "3.2 Алгоритм расчета непредъявленной выручки": 65,
            "3.3 Алгоритм расчета переоценки непредъявленной выручки": 65,
            "Алгоритм признания расходов в налоговом учете по договорам ДЦИ": 69,
            "4 КАРТА ПРОВОДОК": 72,
            "4.1 Для ДЦИ с вариантом отражения «Только БУ»": 4.1,
            "4.1.1 Проводки регламентированного учета при отражении РТУ (тип «Частичное признание»)": 72,
            "4.1.2 Проводки при отражении РТУ с типом «Акт выполненных работ» с видом ДЦИ «Только в БУ»": 73,
            "4.1.3 Проводки при отражении валютных РТУ с типом «Закрывающий акт» с видом ДЦИ «Только БУ»": 75,
            "4.1.4 Финальное актирование договоров ДЦИ в случае остатков по НУ": 76,
            "4.1.5 Проводки корректировки реализации работ с типом «Частичное признание» по увеличению выручки": 77,
            "4.1.6 Проводки корректировки реализации работ с типом «Частичное признание» по уменьшению выручки": 78,
            "4.1.7 Проводки корректировки реализации работ с типом «Акт выполненных работ» по увеличению выручки документа реализации текущего года": 80,
            "4.1.8 Проводки корректировки реализации работ с типом «Акт выполненных работ» по увеличению выручки документа реализации прошлого года": 82,
            "4.1.9 Проводки корректировки реализации работ с типом «Акт выполненных работ» по уменьшению выручки документа реализации текущего года": 84,
            "4.1.10 Проводки корректировки реализации работ с типом «Акт выполненных работ» по уменьшению выручки документа реализации прошлого года": 87,
            "4.1.11 Проводки регламентной операции переоценки валютной непредъявленной выручки по договорам ДЦИ": 89,
            "4.2 Для ДЦИ с видом «БУ, НУ»": 90,
            "4.2.1 Проводки регламентированного учета при отражении РТУ (тип «Частичное признание») с типом ДЦИ «БУ,НУ»": 91,
            "4.2.2 Проводки при отражении РТУ с типом «Акт выполненных работ» с видом ДЦИ «БУ,НУ»": 93,
            "4.2.3 Проводки корректировки реализации работ с типом «Частичное признание» по увеличению выручки": 94,
            "4.2.4 Проводки корректировки реализации работ с типом «Частичное признание» по уменьшению выручки": 95,
            "4.2.5 Проводки корректировки реализации работ с типом «Акт выполненных работ» по увеличению выручки документа реализации текущего года и прошлого года": 97,
            "4.2.6 Проводки корректировки реализации работ с типом «Акт выполненных работ» по уменьшению выручки документа реализации текущего года и прошлого года": 98,
            "4.2.7 Проводки регламентной операции переоценки валютной непредъявленной выручки по договорам ДЦИ": 100,
            "ПРЕДУПРЕЖДЕНИЯ, СООБЩЕНИЯ ОБ ОШИБКАХ": 103
        }
    },
    "Инструкция D-1C1-1.19.01 Поступление ТМЦ на склад предприятия от поставщиков_v18.pdf": {
        "name": "Поступление ТМЦ на склад предприятия от поставщиков, регистрация входящего счета-фактуры",
        "titles": {
            "ПЕРЕЧЕНЬ СОКРАЩЕНИЙ, ТЕРМИНОВ И ОПРЕДЕЛЕНИЙ": 7,
            "ЦЕЛИ И ЗАДАЧИ НАСТОЯЩЕЙ ИНСТРУКЦИИ": 8,
            "1 ОПИСАНИЕ ЭТАПОВ БИЗНЕС-ПРОЦЕССА": 9,
            "1.1 Последовательность этапов бизнес-процесса": 9,
            "1.2 Профили групп доступа": 10,
            "1.3 Описание этапов бизнес-процесса": 12,
            "Проверка соответствия пакета документов по номенклатуре, количеству, оформлению (Кладовщик)": 12,
            "Информирование ответственного за закупку о несоответствии (Кладовщик)": 12,
            "Оформление первичной документации на поступившие ТМЦ (Кладовщик)": 13,
            "1.3.3.1 Аннулирование документа «Приемка ТМЦ».": 22,
            "Регистрация и проведение документов приобретения товаров и услуг (Менеджер по закупкам).": 24,
            "1.3.4.1 Функционал изменения даты курса валюты в валютном документе.": 40,
            "1.3.4.2 Отражение поступления электроэнергии": 42,
            "1.3.4.3.1 Особенности отражения входящих электронных документов на агентское вознаграждение (Менеджер по закупкам, Менеджер ЮЗ ЭДО)": 50,
            "1.3.4.4 Регистрация счета-фактуры": 53,
            "1.3.4.5 Проверка и согласование ОЦО документов приобретения товаров и услуг (Бухгалтер по учету ТМЦ и услуг)": 57,
            "1.3.4.6 Подписание входящего электронного документа приобретения товаров и услуг (Подписант ЮЗ ЭДО)": 63,
            "1.3.4.7 Отражение документов приобретения товаров и услуг регламентированном учете (Бухгалтер по учету ТМЦ и услуг)": 67,
            "Корректировка объектов системы в случае расхождения с первичной документацией (Менеджер по закупкам)": 67,
            "1.3.6.1 Безвозмездное поступление ТМЦ": 68,
            "1.3.6.2 Поступление ТМЦ на ответственное хранение": 73,
            "Оформление ГТД в системе (Бухгалтер по учету ТМЦ)": 75,
            "Списание части пришедшего ТМЦ на стоимость партии (Бухгалтер по учету ТМЦ)": 80,
            "Определение факта прихода документов услуг ТЗР (Менеджер по закупкам)": 86,
            "Регистрация и проведение Доп. расходов в системе (Менеджер по закупкам)": 86,
            "Согласование поступления доп. расходов (Менеджер по закупкам, Менеджер ЮЗ ЭДО, Бухгалтер, Подписант ЮЗ ЭДО)": 96,
            "Отражение Доп. расходов в регл. учете при использовании функционала электронного документооборота (Бухгалтер по учету услуг)": 97,
            "Корректировка Доп. расходов в системе (Менеджер по закупкам)": 97,
            "Отражение ТЗР, которые невозможно распределить на стоимость ТМЦ (Бухгалтер по учету ТМЦ)": 98,
            "Регистрация и проведение корректировки приобретения ( Менеджер по закупкам)": 99,
            "1.3.15.1 Корректировка приобретения до ввода остатков": 106,
            "1.3.15.1.1 Корректировка приобретения до ввода остатков по счетам- фактурам с несколькими документами-основаниями.": 114,
            "1.3.15.1.2 Корректировка приобретения до ввода остатков по счетам- фактурам с несколькими документами-основаниями, особенности использования функционала ЮЗ ЭДО.": 117,
            "1.3.15.4 Подписание входящего электронного документа корректировки приобретения (Подписант ЮЗ ЭДО)": 124
        }
    },
    "Инструкция D-1C1-1.10.04 Учет банковских операций (НФ)_v2 (28) (1).pdf": {
        "name": "Учет банковских операций",
        "titles": {
            "ПЕРЕЧЕНЬ СОКРАЩЕНИЙ, ТЕРМИНОВ И ОПРЕДЕЛЕНИЙ": 7,
            "ЦЕЛИ И ЗАДАЧИ НАСТОЯЩЕЙ ИНСТРУКЦИИ": 8,
            "ОПИСАНИЕ ЭТАПОВ БИЗНЕС-ПРОЦЕССА": 9,
            "1.1 Последовательность этапов бизнес-процесса": 9,
            "1.2 Описание этапов бизнес-процесса": 9,
            "Бухгалтер по учету банковских операций: Проверка документа «Списание безналичных денежных средств»": 9,
            "Казначей, Бухгалтер по учету банковских операций: Получение выписки банка": 23,
            "Бухгалтер по учету банковских операций: Проверка документа «Поступление безналичных денежных средств»": 45,
            "Ответственный за отражение операций эквайринга: Получение данных о платежах через терминалы": 56,
            "Бухгалтер по учету банковских операций: Проверка документа «Эквайринговая операция»": 64,
            "Бухгалтер по учету банковских операций: Сверка оборотов по расчетным счетам за период": 68,
            "БАНКОВСКИЕ ОПЕРАЦИИ В РАМКАХ УЧЕТА ВНУТРИХОЗЯЙСТВЕННЫХ РАСЧЕТОВ": 71,
            "2.1 Постоплата по документу «Реализация товаров и услуг»": 71,
            "2.2 Постоплата по документу «Приобретение товаров и услуг»": 73,
            "2.3 Перечисление налогов за филиал": 75,
            "2.4 Перечисление подотчетному лицу за филиал": 77,
            "2.5 Перечисление с видом операции «Прочее списание» за филиал": 80,
            "2.6 филиал Поступление безналичных ДС с видом операции «Возврат от подотчетника»": 83,
            "Поступление безналичных денежных средств с видом операции «Прочее 2.7 поступление» за филиал": 84,
            "2.8 Постоплата по документу «Приобретение товаров и услуг» по договору филиала": 86,
            "2.9 Поступление оплаты по документу «Поступление безналичных денежных средств» с видом операции «Поступление по депозитам» по договору филиала": 91,
            "2.10 Поступление оплаты по документу «Поступление безналичных денежных средств» с видом операции «Поступление по кредитам и займам полученным» по договору филиала": 92,
            "Поступление оплаты по документу «Поступление безналичных денежных средств» с 2.11 видом операции « Погашение займа контрагентом» по договору филиала": 94,
            "2.12 операции «Оплата по кредитам и займам полученным» по договору филиала": 96,
            "Перечисление по документу «Списание безналичных денежных средств» с видом 2.13 операции «Перечисление на депозиты» по договору филиала": 98,
            "Перечисление по документу «Списание безналичных денежных средств» с видом 2.14 операции «Выдача займа контрагенту» по договору филиала": 99,
            "Перечисление по документу «Списание безналичных денежных средств» с видом ПОРЯДОК ОТРАЖЕНИЯ ОПЕРАЦИЙ С МУЛЬТИВАЛЮТНЫМИ ПЛАТЕЖАМИ": 102,
            "ПОРЯДОК ОТРАЖЕНИЯ ПЛАТЕЖЕЙ В УСЛОВНЫХ ЕДИНИЦАХ": 106,
            "5.2 (USD, EUR) в валюту регламентированного учета (RUB) (Продажа иностранной валюты).": 118,
            "Конвертация валюты из валюты отличной от валюты регламентированного учета ВОЗВРАТ ВАЛЮТЫ": 124,
            "6.1 Возврат аванса от поставщика, где объект расчета Заказ поставщику": 124,
            "6.2 Возврат аванса, где объектом расчетов является сам документ «Списание безналичных ДС»": 130,
            "6.3 «Поступление безналичных ДС»). Высвобождение аванса.": 137,
            "Возврат аванса от поставщика, если изначально была постоплата (объект расчета ОБЕСПЕЧИТЕЛЬНЫЕ ПЛАТЕЖИ": 144,
            "КОРРЕКТИРОВКА СТАТЬИ ДДС ДЛЯ БАНКОВСКИХ ДОКУМЕНТОВ ЗАКРЫТОГО ПЕРИОДА": 149,
            "КОРРЕКТИРОВКА ПРОЕКТА ДЛЯ ДОКУМЕНТА «СПИСАНИЕ БЕЗНАЛИЧНЫХ ДС»": 157,
            "ОБРАБОТКА «ЗАПОЛНЕНИЕ ПЛАТЕЖЕЙ ПО ЗАРПЛАТЕ»": 160,
            "ОБРАБОТКА «ЗАПОЛНЕНИЕ ПЛАТЕЖЕЙ ПО АЛИМЕНТАМ»": 164
        }
    },
    "Инструкция D-1C1-1.10.02 Учет денежных документов в кассе_v2 (НФ).pdf": {
        "name": "Учет денежных документов в кассе",
        "titles": {
            "ПЕРЕЧЕНЬ СОКРАЩЕНИЙ, ТЕРМИНОВ И ОПРЕДЕЛЕНИЙ": 4,
            "Цели и задачи настоящей инструкции": 5,
            "Описание этапов бизнес-процесса": 5,
            "2.1 Последовательность этапов бизнес-процесса": 5,
            "2.2 Описание этапов бизнес-процесса": 5,
            "Бухгалтер по учету кассовых операций: Формирование документа поступление денежных документов": 5,
            "2.2.1.1 Вид операции «Поступление от поставщика»": 8,
            "2.2.1.2 Вид операции «Поступление от подотчетного лица»": 11,
            "2.2.1.3 Вид операции «Прочие поступления денежных документов»": 12,
            "Бухгалтер по учету кассовых операций: Формирование документа выдача денежных документов": 15,
            "2.2.2.1 Вид операции «Выдача подотчетному лицу»": 16,
            "2.2.2.2 Вид операции «Возврат поставщику»": 19,
            "2.2.2.3 Вид операции «Списание на расходы/ активы»": 21,
            "2.2.2.4 Вид операции «Выдача путевок сотрудникам»": 23,
            "2.2.2.5 Вид операции «Перемещение между МОЛ»": 26,
            "Бухгалтер по учету кассовых операций: Формирование отчета по денежным документам": 28,
            "3. ПРЕДУПРЕЖДЕНИЯ, СООБЩЕНИЯ ОБ ОШИБКАХ": 29
        }
    },
    "D-1C1-1.18.03 Реализация продукции товаров работ услуг и прочая реализация_v26.pdf": {
        "name": "Реализация продукции, товаров, работ, услуг и прочая реализация",
        "titles": {
            "ПЕРЕЧЕНЬ СОКРАЩЕНИЙ, ТЕРМИНОВ И ОПРЕДЕЛЕНИЙ": 7,
            "1 ОПИСАНИЕ ЭТАПОВ БИЗНЕС-ПРОЦЕССА": 8,
            "1.1 ПОСЛЕДОВАТЕЛЬНОСТЬ ЭТАПОВ БИЗНЕС-ПРОЦЕССА": 8,
            "1.2 ПРОФИЛИ ГРУПП ДОСТУПА": 9,
            "1.3 ОПИСАНИЕ ЭТАПОВ БИЗНЕС-ПРОЦЕССА": 12,
            "1.3.1 Отражение обязательств по договору обеспечению полученные (Бухгалтер по учету банковских операций)": 12,
            "1.3.4 Определение момента перехода права собственности (Специалист)": 13,
            "1.3.5 Определение типа документооборота (Менеджер по продажам)": 13,
            "1.3.6 Формирование реализации товаров и услуг без перехода права собственности (ЭДО) (Ответственные лица предприятия)": 13,
            "1.3.7 Отражение реализации без перехода права собственности (Менеджер по продажам)": 14,
            "1.3.11 Формирование реализации полуфабрикатов": 36,
            "1.3.12 Отражение реализации ПО и начисление доходов будущих периодов": 37,
            "1.3.13 Определение типа документооборота (Менеджер по продажам)": 38,
            "1.3.14 Отражение реализации товаров и услуг в системе (ЭДО) (Ответственные лица предприятия)": 38,
            "1.3.15 Отражение реализации в регламентированном учете (документ «Реализация товаров и услуг»)": 46,
            "1.3.17 Определение типа документооборота (Менеджер по продажам)": 94,
            "1.3.18 Отражение корректировки (ЭДО) (Ответственные лица предприятия)": 95,
            "1.3.19 Отображение корректировки реализации (Бухгалтер по учету ТМЦ)": 99,
            "1.3.20 Корректировка реализации до ввода остатков (Бухгалтер по учету ТМЦ)": 111,
            "1.3.21 Заполнение префикса в счет-фактуре": 119,
            "1.3.22 Особое формирование номеров счетов-фактур выданных": 125,
            "1.3.23 Расчеты по централизованным договорам": 128,
            "1.3.24 Зачет оплаты (распределение платежей автоматически/вручную)": 135,
            "1.3.25 Формирование реализации работ": 138,
            "1.3.26 Замена подразделения в «Заказе клиента» на подразделение из группы фин. Учета": 139
        }
    },
    "Инструкция D-1C1-1.10.10 Учет результатов инвентаризации объектов учета ВНА, РБП, ДБП.pdf": {
        "name": "Учет результатов инвентаризации объектов учета ВНА, РБП, ДБП",
        "titles": {
            "ПЕРЕЧЕНЬ СОКРАЩЕНИЙ, ТЕРМИНОВ И ОПРЕДЕЛЕНИЙ": 5,
            "ЦЕЛИ И ЗАДАЧИ НАСТОЯЩЕЙ ИНСТРУКЦИИ": 6,
            "ОПИСАНИЕ ЭТАПОВ БИЗНЕС-ПРОЦЕССА": 7,
            "1.1 Последовательность этапов бизнес-процесса": 7,
            "1.2 Профили групп доступа": 8,
            "1.3 Описание этапов бизнес-процесса": 8,
            "Определение фактических остатков ВНА": 8,
            "Бухгалтер по учету операций с ВНА: Определение вида объекта ВНА": 9,
            "Бухгалтер по учету операций с ВНА: Регистрация в системе результатов инвентаризации, создание документов «Инвентаризация ОС», «Инвентаризация НМА»": 9,
            "Бухгалтер по учету операций с ВНА: Определение результатов инвентаризации ВНА": 10,
            "1.3.4.1 Создание документа «Инвентаризация ОС»": 10,
            "1.3.4.2 Создание документа Инвентаризация НМА": 22,
            "1.3.4.3 Создание документа Инвентаризация РБП_ДБП": 29,
            "1.3.4.4 Пакетное формирование инвентаризационных описей по ВНА": 35,
            "1.3.4.5 Проверочный отчет инвентаризации": 40,
            "Бухгалтер учету операций с ВНА: Определение вида неучтенного объекта ВНА": 42,
            "Бухгалтер по учету операций с ВНА: Оприходование излишков, выявленных при инвентаризации": 42,
            "2 ПРЕДУПРЕЖДЕНИЯ, СООБЩЕНИЯ ОБ ОШИБКАХ": 44
        }
    },
    "Инструкция D-1C1-1.22.01 Приложение 5 Обработка по корректировке графиков.pdf": {
        "name": "Формирование графиков платежей, поступления ДС, поставок, отгрузок",
        "titles": {
            "ОБРАБОТКА «КОРРЕКТИРОВКА ГРАФИКОВ (ГА)»": 3
        }
    },
    "Инструкция D-1C1-1.01.07 Формирование заявки на платеж.pdf": {
        "name": "Формирование заявки на платеж",
        "titles": {
            "ПЕРЕЧЕНЬ СОКРАЩЕНИЙ, ТЕРМИНОВ И ОПРЕДЕЛЕНИЙ": 7,
            "1.2 Описание этапов бизнес-процесса": 7,
            "Инициатор платежа: Формирование заявки на оплату": 7,
            "1.2.1.1 Документ «Заявка на оплату» с видом операции «Оплата поставщику»": 11,
            "1.2.1.2 Документ «Заявка на оплату» с видом операции «Оплата поставщику» (в рамках государственного заказа)": 28,
            "1.2.1.3 Документ «Заявка на оплату» с видом операции «Уплата налога»": 31,
            "1.2.1.4 Документ «Заявка на оплату» с видом операции «Уплата налога» (единый налоговый платеж)": 39,
            "1.2.1.5 Документ «Заявка на оплату» с видом операции «Оплата поставщику» (налоговый агент)": 41,
            "1.2.1.6 Документ «Заявка на оплату» с видом операции «Перечисление подотчетному лицу»": 43,
            "1.2.1.7 Документ «Заявка на оплату» с видом операции «Прочее списание»": 52,
            "1.2.1.8 Документ «Заявка на оплату» с видом операции «Платеж по кредиту или займу»": 63,
            "1.2.1.9 Документ «Заявка на оплату» с видом операции «Оплата арендодателю».": 67,
            "1.2.1.10 Документ «Заявка на оплату» с видом операции «Оплата поставщику» - валюта документа отличная от валюты регламентированного учета": 74,
            "1.2.1.11 Документ «Заявка на оплату» с видом операции «Оплата поставщику» - мультивалютная заявка на оплату.": 86,
            "1.2.1.12 Документ «Заявка на оплату» с видом операции «Оплата поставщику» - заявка в условных единицах.": 100,
            "1.2.1.13 Документ «Распоряжения на перемещение ДС»": 107,
            "1.3 Автоматически: Формирование печатной формы «Образец платежного поручения»": 110,
        }
    },
    "Инструкция D-1C1-1.10.19 Формирование налоговой декларации по НП-v3.pdf": {
        "name": "Формирование налоговой декларации по налогу на прибыль",
        "titles": {
            "ПЕРЕЧЕНЬ СОКРАЩЕНИЙ, ТЕРМИНОВ И ОПРЕДЕЛЕНИЙ": 4,
            "ЦЕЛИ И ЗАДАЧИ НАСТОЯЩЕЙ ИНСТУКЦИИ": 5,
            "1 ОПИСАНИЕ ЭТАПОВ БИЗНЕС-ПРОЦЕССА": 6,
            "1.1 Последовательность этапов бизнес-процесса": 6,
            "1.2 Профили групп доступа": 7,
            "1.3 Описание этапов бизнес-процессов": 7,
            "1.3.1 Методологические основы": 7,
            "1.3.1.1 Общий порядок учета": 7,
            "1.3.1.2 Порядок исчисления и уплаты налога": 20,
            "1.3.1.3 Сроки представления налоговых деклараций": 22,
            "1.3.2 Общий порядок заполнения декларации": 22,
            "1.3.2.1 Состав декларации по налогу на прибыль": 23,
            "1.3.2.2 Обязательные к заполнению листы": 25,
            "1.3.3 Нормативно-справочная информация": 36,
            "1.3.3.1 Настройки по учету налога на прибыль": 36,
            "1.3.3.2 Справочник «Объекты эксплуатации»": 40,
            "1.3.3.3 Справочник «Номенклатура»": 41,
            "1.4 Формирование декларации по налогу на прибыль.": 41,
            "1.4.1 Создание отчета": 42,
            "1.4.2 Функциональные кнопки в шапке отчета": 44,
            "1.4.3 Заполнение созданной декларации": 50,
            "1.4.4 Настройка расчета параметров налоговой декларации": 56,
            "1.4.5 Помощник заполнения декларации по налогу на прибыль": 60,
            "1.4.6 Расшифровка к налоговой декларации": 67,
            "1.4.7 «Склейка» налоговой декларации": 71,
            "1.4.8 Формирование корректировочной декларации по НП": 74,
            "1.4.9 Учет корректировок налога на прибыль. Отражение доходов и расходов прошлых периодов в текущей Декларации по налогу на прибыль": 75,
            "1.5 Регистры налогового учета по налогу на прибыль": 78,
            "2 ПРЕДУПРЕЖДЕНИЯ, СООБЩЕНИЯ ОБ ОШИБКАХ": 83,
        }
    },
    "D-1C1-1.05.01 Планирование работы ТС.pdf": {
        "name": "Планирование работы ТС",
        "titles": {
            "1. ЦЕЛИ И ЗАДАЧИ НАСТОЯЩЕЙ ИНСТРУКЦИИ": 4,
            "2. ОПИСАНИЕ ЭТАПОВ БИЗНЕС-ПРОЦЕССА": 5,
            "1.1 ПОСЛЕДОВАТЕЛЬНОСТЬ ЭТАПОВ БИЗНЕС-ПРОЦЕССА": 5,
            "1.2 ОПИСАНИЕ ЭТАПОВ БИЗНЕС-ПРОЦЕССА": 5,
            "2.1.1 Создание сценария планирования работы ТС по заказам": 5,
            "2.1.2 Создание документа по планированию работы ТС по заказам": 7,
        }
    },
    "Инструкция D-1C1-1.13.01 Ведение договоров на закупку ТМЦ работ и услуг (НФ) (3).pdf": {
        "name": "Ведение договоров на закупку ТМЦ, работ и услуг",
        "titles": {
            "ПЕРЕЧЕНЬ СОКРАЩЕНИЙ, ТЕРМИНОВ И ОПРЕДЕЛЕНИЙ": 4,
            "ЦЕЛИ И ЗАДАЧИ НАСТОЯЩЕЙ ИНСТРУКЦИИ": 5,
            "1 ОПИСАНИЕ ЭТАПОВ БИЗНЕС-ПРОЦЕССА": 6,
            "1.1 Последовательность этапов бизнес-процесса": 6,
            "1.2 Профили групп доступа": 6,
            "1.3 Описание этапов бизнес-процесса": 7,
            "1.3.1 Регистрация договора в системе, в том числе договоров с обеспечением (Менеджер по закупкам)": 7,
            "1.3.1.1 Справочник «Договоры с контрагентами»": 7,
            "1.3.1.2 Версии договора": 16,
            "1.3.1.3 Дополнительное соглашение к договору": 19,
            "1.3.1.4 Особенности работы с договором ЭДО": 20,
            "1.3.2 Создание банковского счета (Казначей / Бухгалтер по учету ТМЦ / Бухгалтер по учету услуг)": 21,
            "ПРЕДУПРЕЖДЕНИЯ, СООБЩЕНИЯ ОБ ОШИБКАХ": 28
        }
    },
    "Инструкция Списание НДС на расходы.pdf": {
        "name": "Списание НДС на расходы",
        "titles": {
            "Списание НДС на расходы": 4,
            "1.1. Закладка «Списания НДС»": 5,
            "1.2. Закладка «Входящий НДС, не подтвержденный счетом-фактурой»": 8,
            "1.3. Закладка «Входящий НДС, по командировочным расходам»": 11,
            "1.4. Закладка «Входящий НДС, по не принимаемым расходам в НУ»": 12,
            "1.4.1. Регистрация статей-исключений": 13,
            "1.4.2. Вкладка «Текущие списания»": 15,
            "1.4.3. Вкладка «Списания с остатков»": 23,
            "1.4.4. Разделение документов для исключения ошибок при списании НДС": 26,
        }
    },
    "Инструкция D-1C1-1.10.14  Учет НДС_v7.pdf": {
        "name": "Учет НДС",
        "titles": {
            "ПЕРЕЧЕНЬ СОКРАЩЕНИЙ, ТЕРМИНОВ И ОПРЕДЕЛЕНИЙ": 7,
            "ЦЕЛИ И ЗАДАЧИ НАСТОЯЩЕЙ ИНСТУКЦИИ": 8,
            "1. ОПИСАНИЕ ЭТАПОВ БИЗНЕС-ПРОЦЕССА": 9,
            "1.1. Последовательность этапов бизнес-процесса": 9,
            "1.2. Профили групп доступа": 11,
            "1.3. Описание этапов бизнес-процесса": 12,
            "1.3.1. Общие настройки системы влияющие на учет НДС": 12,
            "1.3.2. Настройка учетной политики организации": 23,
            "1.3.3. Настройка направлений деятельности и статей расходов": 33,
            "1.3.4. Настройка номенклатуры": 36,
            "1.3.5. Помощник по учету НДС": 39,
            "1.3.6. Рабочее место «Авансы полученные» - Бухгалтер по учету налоговых обязательств и формированию отчетности (НДС)": 49,
            "1.3.7. Рабочее место «Авансы выданные» - Бухгалтер по учету налоговых обязательств и формированию отчетности (НДС)": 64,
            "1.3.8. Подтверждение нулевой ставки НДС - Бухгалтер по учету налоговых обязательств и формированию отчетности (НДС).": 69,
            "1.3.9. Блокировка вычета НДС - Бухгалтер по учету налоговых обязательств и формированию отчетности (НДС)": 72,
            "1.3.10. Корректировка вида деятельности НДС - Бухгалтер по учету налоговых обязательств и формированию отчетности (НДС)": 79,
            "1.3.11. Корректировка назначения товаров - Бухгалтер по учету налоговых обязательств и формированию отчетности (НДС)": 82,
            "1.3.13. Импорт - Бухгалтер по учету налоговых обязательств и формированию отчетности (НДС)": 93,
            "1.3.14. Экспорт сырьевой - Бухгалтер по учету налоговых обязательств и формированию отчетности (НДС)": 110,
            "1.3.15. Неотфактурованная поставка Бухгалтер по учету налоговых обязательств и формированию отчетности (НДС)": 118,
            "1.3.16. Приобретение товаров и услуг под деятельность «Не облагаемую НДС» - Бухгалтер по учету налоговых обязательств и формированию отчетности (НДС).": 130,
            "1.3.17. Начисление НДС по СМР - Бухгалтер по учету налоговых обязательств и формированию отчетности (НДС)": 132,
            "1.3.18. Определение 5% барьера": 153,
            "1.3.19. Распределение НДС": 166,
            "1.3.20. Отражение НДС по ВНА": 175,
            "1.3.21. Списание НДС на расходы": 183,
            "1.3.22. Учет НДС по ДЦИ": 185,
            "1.3.23. Формирование СФ (БСО) в авансовых отчетах по электронным документам, приобретенных через агента.": 189,
            "1.3.24. Настройки документов для раздельного учета НДС при поступлении ТМЦ и услуг": 225,
            "2. ПРЕДУПРЕЖДЕНИЯ, СООБЩЕНИЯ ОБ ОШИБКАХ": 227,
        }
    },
    "Инструкция D-1C1-1.20.02 Анализ взаиморасчетов по закупкам (1).pdf": {
        "name": "Анализ взаиморасчетов по закупкам",
        "titles": {
            "Цели и задачи настоящей инструкции": 6,
            "Описание этапов бизнес-процесса": 7,
            "2.1 Последовательность этапов бизнес-процесса": 7,
            "2.2 Описание этапов бизнес-процесса": 7,
            "Специалист планово-экономического отдела: Анализ взаиморасчетов и необходимости внесения изменений в ее структуру": 7,
            "2.2.1.1 Формирование отчета «Задолженность поставщикам по срокам»": 7,
            "2.2.1.2 Формирование отчета «Задолженность поставщикам»": 9,
            "2.2.1.3 Формирование отчета «Ведомость расчетов с поставщиками»": 11,
            "2.2.1.4 Формирование отчета «Сводная ведомость расчетов»": 13,
            "2.2.1.5 Формирование отчета «Отчет по операциям со связанными сторонами»": 15,
            "2.2.1.6 Формирование отчета «Выполнение условий по договорам контрагентов»": 17,
        }
    },
    "Инструкция D-1C1-1.10.41 Учет операций при закрытии периода (НФ).pdf": {
        "name": "Учет операций при закрытии периода",
        "titles": {
            "2.1. Последовательность этапов бизнес-процесса": 7,
            "2.2. Описание этапов бизнес-процесса": 20,
            "2.2.2.2. Бухгалтер, ответственный за участок учета: Изменение первичных документов за отчетный период": 27,
            "2.2.2.3. Бухгалтер, ответственный за участок учета: Перепроведение первичных документов": 35,
            "2.2.4.2 Бухгалтер, ответственный за участок учета ВНА и НИОКР: Начисление амортизации ОС, НМА": 37,
            "2.2.4.3 Бухгалтер, ответственный за участок учета ВНА и НИОКР: Признание в налоговом учете арендных платежей (Регл. учет)": 42,
            "2.2.4.4 Бухгалтер, ответственный за участок учета ВНА и НИОКР: Ежемесячный расчет земельного, имущественного и транспортного налогов": 43,
            "2.2.4.5 Бухгалтер, ответственный за участок учета ТМЦ: Погашение стоимости ТМЦ в эксплуатации": 47,
            "2.2.4.6 Бухгалтер, ответственный за участок учета ВНА и НИОКР: Расчет земельного, имущественного и транспортного налогов (ежеквартально)": 48,
            "2.2.4.7 Бухгалтер, ответственный за участок учета ВНА и НИОКР: Формирование движений ОС и НМА": 51,
            "2.2.4.8 Бухгалтер, ответственный за участок учета ВНА и НИОКР: Начисление процентов по аренде": 53,
            "2.2.4.9 Бухгалтер, ответственный за участок учета ВНА и НИОКР: Закрытие расходов от выбытия ОС": 57,
            "2.2.4.10 Бухгалтер, ответственный за участок учета ВНА и НИОКР: Переоценка взаиморасчетов по аренде": 58,
            "2.2.4.11 Бухгалтер, ответственный за формирование бухгалтерской отчетности: Формирование резерва по сомнительным долгам": 58,
            "2.2.5. Бухгалтер, ответственный за участок учета затрат и формирование себестоимости: Распределение производственных затрат": 66,
        }
    },
    "Инструкция D-1C1-1.10.47 Учет резервов по сомнительным долгам.pdf": {
        "name": "Учет резервов по сомнительным долгам",
        "titles": {
            "Цели и задачи настоящей инструкции": 8,
            "Описание этапов бизнес-процесса": 9,
            "2.1 Последовательность этапов бизнес-процесса": 9,
            "2.2 Описание этапов бизнес-процесса": 9,
            "2.2.1 Экономист: Предварительные настройки системы": 11,
            "2.2.2 Настройка видов условий в справочнике «Договоры контрагентов»": 21,
            "2.2.2.1 Заполнение реквизита «Вид условия отгрузки»": 23,
            "2.2.2.2 Заполнение реквизита «Вид условия оплаты»": 30,
            "2.3 Бухгалтер по сверке взаимных расчетов: Формирование резерва по сомнительным долгам": 40,
            "2.3.1 Заполнение реквизитов в документе «Начисление и списание резервов по сомнительным долгам»": 45,
            "2.4 Виды задолженностей": 55,
            "2.5 Начисление задолженности по косвенным налогам и сборам": 57,
            "2.6 Случай неначисления резерва": 60,
            "Бухгалтер по сверке взаимных расчетов: Восстановление резерва в случае оплаты": 65,
            "2.9 Бухгалтер по сверке взаимных расчетов: Восстановление резерва в разделах учета": 69,
            "2.10 Бухгалтер по сверке взаимных расчетов: Списание задолженности, отраженной за балансом": 71,
            "2.11 Формирование отчетов": 73,
        }
    },
    "Инструкция администратора Описание настроек подсистемы бюджетирования для АО Гринатом.pdf": {
        "name": "Описание настроек подсистемы бюджетирования для АО Гринатом",
        "titles": {
            "Перечень сокращений, терминов и определений": 6,
            "1. Цели и задачи настоящей инструкции": 7,
            "2. Последовательность и описание этапов бизнес-процесса": 7,
            "2.1. Последовательность выполнения бизнес-процессов": 7,
            "2.2. Описание этапов бизнес-процесса": 7,
            "2.2.1. Администратор: установка параметров плана": 9,
            "2.2.2. Администратор: настройка структуры оперативного планирования": 9,
            "2.2.3. Администратор: работа с вариантами настроек операционного плана": 13,
            "2.2.4. Администратор: установка параметров лимитирования": 16,
            "2.2.5. Администратор: настройка параметров лимитирования статей бюджетов": 18,
            "2.2.6. Администратор: настройка функциональных опций": 20,
            "2.2.7. Администратор: настройка получения факта по бюджетам": 21,
            "2.2.8. Администратор: настройка регистра сведений «Правила заполнения объектов»": 24,
            "2.2.9. форме корректировки лимитов.": 26,
            "Администратор: Описание настройки заполнения полей подписантов в печатной Приложение А": 28,
            "А-1. Для варианта «ОФД из документа «Списание безналичных денежных средств»": 28,
            "А-2. Для варианта «ОФД из документа «Поступление безналичных денежных средств»": 29,
            "А-3. Для варианта «ОФД из документа «Расчет курсовых разниц»": 30,
            "Приложение Б": 31,
            "Б-1. Для варианта «ОФД из документа «Списание безналичных денежных средств»": 31,
            "Б-2. Для варианта «ОФД из документа «Поступление безналичных денежных средств»": 32,
            "Б-3. Для варианта «ОФД из документа «Расчет курсовых разниц»": 32,
        }
    },
    "Инструкция Настройки регламентированного учета.pdf": {
        "name": "Инструкция пользователей ОЦО «Настройки регламентированного учета»",
        "titles": {
            "Цели и задачи настоящей инструкции": 6,
            "1 Описание настроек": 7,
            "2.1 Функциональная опция «Регламентированный учет»": 7,
            "2.2 План счетов": 10,
            "2.3 Общие классификаторы регламентированного учета": 12,
            "2.4 Учетная политика": 13,
            "2.5 Установка ставок налогов": 14,
            "2.5.1 Ставки налога на прибыль": 14,
            "2.5.2 Ставки налога на имущество": 15,
            "2.6 Настройки счетов учета для отражения в документах": 15,
            "2.6.1 Рабочее место отражения в регламентированном учете": 16,
            "2.6.2 Настройка групп финансового учета расчетов": 21,
            "2.6.3 Настройка групп финансового учета номенклатуры": 23,
            "2.6.4 Настройка групп финансового учета внеоборотных активов": 27,
            "2.6.5 Настройка статей расходов": 29,
            "2.6.6 Настройка статей доходов": 32,
            "2.6.7 Настройка видов расходов предстоящих периодов": 34,
            "2.6.8 Настройка банковских счетов": 35,
        }
    },
    "Инструкция D-1C1-1.27.10 Учет договоров обеспечения и платежей по обеспечению.pdf": {
        "name": "Учет обеспечения кредита (займа) «Учет договоров обеспечения выданных/полученных и платежи по обеспечению",
        "titles": {
            "СОДЕРЖАНИЕ ПЕРЕЧЕНЬ СОКРАЩЕНИЙ, ТЕРМИНОВ И ОПРЕДЕЛЕНИЙ": 4,
            "ЦЕЛИ И ЗАДАЧИ НАСТОЯЩЕЙ ИНСТУКЦИИ": 5,
            "1 ОПИСАНИЕ ЭТАПОВ БИЗНЕС-ПРОЦЕССА": 6,
            "1.1 Последовательность этапов бизнес-процесса": 6,
            "1.2 Профили групп доступа": 6,
            "1.3 Описание этапов бизнес-процесса": 7,
            "Определение вида учета суммы гарантий/поручительства (Бухгалтер по учету кредитов и займов)": 7,
            "Отражение суммы обеспечения при возникновении обязательств (Начисление обязательств) (Бухгалтер по учету кредитов и займов)": 7,
            "Корректировка введенных данных при необходимости (Изменение обязательств) (Бухгалтер по учету кредитов и займов)": 13,
            "Списание суммы обеспечений при погашении обязательств (Списание обязательств) (Бухгалтер по учету кредитов и займов)": 15,
            "Оперативная проверка введенных данных по обязательствам (Бухгалтер по учету кредитов и займов)": 18,
            "Отражение обеспечительных платежей, входящих/исходящих в ИС": 19,
        }
    }
}

In [612]:
titles_all_json

{'Инструкция Интеграция с ЕОСДО v5.pdf': {'name': 'Инструкция пользователей «Интеграция 1С ERP 2.0: Цифровой Росатом с ЕОСДО»',
  'titles': {'СОДЕРЖАНИЕ ПЕРЕЧЕНЬ СОКРАЩЕНИЙ, ТЕРМИНОВ И ОПРЕДЕЛЕНИЙ': 4,
   'ЦЕЛИ И ЗАДАЧИ НАСТОЯЩЕЙ ИНСТУКЦИИ': 5,
   'Описание интеграционных потоков и их с взаимосвязи с функциональными бизнес- процессами.': 6,
   '1.1 Интеграционный сценарий «Система электронного архива (СЭА)» (10-й сценарий ЕОСДО)': 8,
   '1.2 Интеграционный сценарий «Массовая выгрузка документов из ЕОСДО»': 10,
   '1.3 Передача платежных поручений (мемориальных ордеров) в ЕОСДО': 10,
   'Описание этапов интеграции': 12,
   '2.1 Интеграционный сценарий «Система электронного архива (СЭА)» (10-й сценарий ЕОСДО)': 12,
   'Предприятие: регистрация первичного документа и проведение в управленческом учете': 12,
   'Предприятие: Отправка пакета первичной документации посредством Captiva': 18,
   'Предприятие / Бухгалтер: Проверка сделки в ЕОСДО': 23,
   'Предприятие / Бухгалтер: Сверка первичны

In [615]:
with open("titles_all.json", "w") as outfile: 
    json.dump(titles_all_json, outfile, ensure_ascii=False)

In [127]:
files = glob.glob("../../data/atomic/pdf/*")

In [600]:
idx = 29
files[idx].split("/")[-1]

'Инструкция D-1C1-1.27.10 Учет договоров обеспечения и платежей по обеспечению.pdf'

In [601]:
loader = UnstructuredPDFLoader(
    file_path=files[idx],
    mode="elements",
    
    strategy="fast",
    # hi_res_model_name="yolox",
    languages=["rus", "eng"],

    include_page_breaks=False,
    include_metadata=True,
    infer_table_structure=True,

    chunking_strategy="basic",  # "basic" Produces a sequence of `CompositeElement`, `Table`, and `TableChunk` elements (chunks).
    max_characters=10_000_000,
    new_after_n_chars=10_000_000,
    combine_text_under_n_chars=10_000_000,
)
documents = loader.load()

In [602]:
text_docs = filter_text_documents(documents)

In [603]:
text = text_docs[0].page_content

In [604]:
text_splitted = [line for line in text.splitlines() if line]

In [605]:
n_remain = 3
remain_cnt = n_remain
started_titles = False
raw_titles = []
for line_idx, line in enumerate(text_splitted[:]):
    if started_titles and remain_cnt <= 0:
        break
    if re.search(r"\.{2,}", line):
        started_titles = True
        remain_cnt = n_remain
        raw_titles.append(line)
        continue
    if started_titles:
        raw_titles.append(line)
        remain_cnt -= 1
raw_titles = raw_titles[:-n_remain]

In [606]:
print(line_idx)

33


In [607]:
title_lines = []
_line = ""
for line_idx, line in enumerate(raw_titles):
    if re.search(r"\d$", line):
        title_lines.append(_line + " " + line)
        _line = ""
    else:
        _line = _line + " " + line
title_lines = [line.strip() for line in title_lines]

In [608]:
for title_line in title_lines:
    title_line_splitted = re.split(r"\.{2,}", title_line)
    print(f"\"{title_line_splitted[0].strip()}\": {title_line_splitted[-1].strip()},")

"СОДЕРЖАНИЕ ПЕРЕЧЕНЬ СОКРАЩЕНИЙ, ТЕРМИНОВ И ОПРЕДЕЛЕНИЙ": 4,
"ЦЕЛИ И ЗАДАЧИ НАСТОЯЩЕЙ ИНСТУКЦИИ": 5,
"1 ОПИСАНИЕ ЭТАПОВ БИЗНЕС-ПРОЦЕССА": 6,
"1.1 Последовательность этапов бизнес-процесса": 6,
"1.2 Профили групп доступа": 6,
"1.3 Описание этапов бизнес-процесса": 7,
"Определение вида учета суммы гарантий/поручительства (Бухгалтер по учету кредитов и займов)": 7,
"Отражение суммы обеспечения при возникновении обязательств (Начисление обязательств) (Бухгалтер по учету кредитов и займов)": 7,
"Корректировка введенных данных при необходимости (Изменение обязательств) (Бухгалтер по учету кредитов и займов)": 13,
"Списание суммы обеспечений при погашении обязательств (Списание обязательств) (Бухгалтер по учету кредитов и займов)": 15,
"Оперативная проверка введенных данных по обязательствам (Бухгалтер по учету кредитов и займов)": 18,
"Отражение обеспечительных платежей, входящих/исходящих в ИС": 19,


In [560]:
print("\n".join([txt for txt in text_splitted]))

а
Проект D-1C1-1 «Развитие унифицированной отраслевой платформы «1С ERP 2.0: ЦИФРОВОЙ РОСАТОМ»
Инструкция пользователей «D-1C1- 1.10.41 Учет операций при закрытии периода»
Дата
12.04.2021
13.04.2021
21.04.2021
16.11.2022
26.04.2024
Автор
Версия
Версия
1
2
3
4
5
АО «Гринатом»
4
Инструкция пользователей D-1C1-1.10.41 Учет операций при закрытии периода
ЛИСТ ИЗМЕНЕНИЙ
Краткое описание изменений Разработка первой версии документа Разработка второй версии документа Разработка третьей версии документа Разработка четвертой версии документа Добавлен п. 2.2.2.4 Операции по подотчетным лицам Добавлен п. 2.2.3.1. Актуализация движения документов по данным взаиморасчетов Выделен п. 2.2.3.2. Переоценка денежных средств и финансовых инструментов Скорректирован п. 2.2.4.11. Формирование резерва по сомнительным долгам Скорректирован п. 2.2.8.4. Расчет курсовых разниц
ФИО
Колганова Н.Л.
Колганова Н.Л.
Колганова Н.Л.
Цветкова С.С.
Докучаева А.В.
Страница
2
Инструкция пользователей «D-1C1-1.10.41 Учет опе